In [ ]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import requests

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
def chunks(lst, n):
  for i in range(0, len(lst),n):
    yield lst[i:i+n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Tinker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
print(symbol_strings)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS', 'GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
  batch_api = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
  response = requests.get(batch_api)
  response.raise_for_status()
  if response.status_code != 204:
    data = response.json()
  for symbol in symbol_string.split(','):
    final_dataframe = final_dataframe.append(
        pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['peRatio'],'N/A'],index = my_columns),ignore_index = True)
    
final_dataframe



,Tinker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.79,28.6,N/A
1,AAL,14.99,-3.9,N/A
2,AAP,188.63,20.9,N/A
3,AAPL,150.75,24.46,N/A
4,ABBV,156.53,22.1,N/A
...,...,...,...,...
496,YUM,120.08,22.29,N/A
497,ZBH,103.32,105.62,N/A
498,ZBRA,304.61,19.99,N/A
499,ZION,51.65,5.33,N/A


In [ ]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Tinker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.16,2.28,N/A
1,EBAY,44.50,2.34,N/A
2,AMZN,114.90,2.78,N/A
3,NUE,111.49,4,N/A
4,COF,106.23,4.14,N/A
5,PVH,59.12,4.15,N/A
6,F,11.77,4.17,N/A
7,SYF,31.03,4.25,N/A
8,KSS,28.00,4.42,N/A
9,AIG,51.28,4.46,N/A


In [ ]:
def portfolio_input():
  global portfolio_size
  portfolio_size = input("Enter the value of your portfolio:")

  try:
    val = float(portfolio_size)
  except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
portfolio_input()

Enter the value of your portfolio:2123


In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)
for i in range(0, len(final_dataframe['Tinker'])-1):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])

final_dataframe

42.46


,Tinker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.16,2.28,1
1,EBAY,44.50,2.34,0
2,AMZN,114.90,2.78,0
3,NUE,111.49,4,0
4,COF,106.23,4.14,0
5,PVH,59.12,4.15,0
6,F,11.77,4.17,3
7,SYF,31.03,4.25,1
8,KSS,28.00,4.42,1
9,AIG,51.28,4.46,0
